# Projectile Motion with Air Resistance

## Introduction

Projectile motion is a fundamental problem in classical mechanics. While the idealized case without air resistance admits an elegant analytical solution, real-world projectiles experience drag forces that significantly alter their trajectories. This notebook explores projectile motion under the influence of quadratic air resistance, comparing numerical solutions to the ideal (vacuum) case.

## Theoretical Background

### Equations of Motion

Consider a projectile of mass $m$ moving through air. The forces acting on it are:

1. **Gravity**: $\vec{F}_g = -mg\hat{j}$
2. **Air Resistance (Drag)**: $\vec{F}_d = -\frac{1}{2}\rho C_d A |\vec{v}|\vec{v}$

where:
- $g$ is the gravitational acceleration ($\approx 9.81$ m/s²)
- $\rho$ is the air density ($\approx 1.225$ kg/m³ at sea level)
- $C_d$ is the drag coefficient (dimensionless, $\approx 0.47$ for a sphere)
- $A$ is the cross-sectional area of the projectile
- $\vec{v}$ is the velocity vector

### Drag Force Model

The quadratic drag force can be written as:

$$\vec{F}_d = -b|\vec{v}|\vec{v}$$

where $b = \frac{1}{2}\rho C_d A$ is the drag coefficient parameter.

### Newton's Second Law

Applying Newton's second law, we obtain the coupled differential equations:

$$m\frac{dv_x}{dt} = -bv\cdot v_x$$

$$m\frac{dv_y}{dt} = -mg - bv\cdot v_y$$

where $v = \sqrt{v_x^2 + v_y^2}$ is the speed.

### Dimensionless Form

Dividing by mass $m$:

$$\frac{dv_x}{dt} = -\frac{b}{m}v\cdot v_x = -\gamma v\cdot v_x$$

$$\frac{dv_y}{dt} = -g - \gamma v\cdot v_y$$

where $\gamma = b/m$ is the drag coefficient per unit mass.

### Terminal Velocity

When a projectile falls vertically with drag, it reaches terminal velocity $v_t$ when the drag force balances gravity:

$$mg = bv_t^2$$

$$v_t = \sqrt{\frac{mg}{b}} = \sqrt{\frac{g}{\gamma}}$$

This provides a convenient way to parameterize the drag: $\gamma = g/v_t^2$.

### Ideal Case (No Air Resistance)

For comparison, the analytical solution without drag is:

$$x(t) = v_0\cos(\theta)\cdot t$$

$$y(t) = v_0\sin(\theta)\cdot t - \frac{1}{2}gt^2$$

with range $R = \frac{v_0^2\sin(2\theta)}{g}$ and maximum height $H = \frac{v_0^2\sin^2(\theta)}{2g}$.

## Numerical Implementation

We solve the system of ODEs using SciPy's `solve_ivp` with the RK45 (Runge-Kutta) method. The state vector is $\vec{y} = [x, y, v_x, v_y]^T$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Physical constants
g = 9.81  # gravitational acceleration (m/s^2)

def projectile_with_drag(t, state, gamma):
    """
    Right-hand side of the ODE system for projectile motion with quadratic drag.
    
    Parameters:
    -----------
    t : float
        Time (not used explicitly, but required by solve_ivp)
    state : array-like
        State vector [x, y, vx, vy]
    gamma : float
        Drag coefficient per unit mass (1/m)
    
    Returns:
    --------
    derivatives : list
        Time derivatives [dx/dt, dy/dt, dvx/dt, dvy/dt]
    """
    x, y, vx, vy = state
    v = np.sqrt(vx**2 + vy**2)  # speed
    
    # Equations of motion
    dxdt = vx
    dydt = vy
    dvxdt = -gamma * v * vx
    dvydt = -g - gamma * v * vy
    
    return [dxdt, dydt, dvxdt, dvydt]

def projectile_no_drag(t, v0, theta):
    """
    Analytical solution for projectile motion without air resistance.
    
    Parameters:
    -----------
    t : array-like
        Time array
    v0 : float
        Initial speed (m/s)
    theta : float
        Launch angle (radians)
    
    Returns:
    --------
    x, y : arrays
        Position coordinates
    """
    x = v0 * np.cos(theta) * t
    y = v0 * np.sin(theta) * t - 0.5 * g * t**2
    return x, y

# Event function to detect when projectile hits the ground
def hit_ground(t, state, gamma):
    return state[1]  # y coordinate

hit_ground.terminal = True
hit_ground.direction = -1  # Only trigger when y is decreasing

print("Functions defined successfully.")

## Simulation Parameters

We simulate a projectile (e.g., a baseball) with the following parameters:

In [ ]:
# Initial conditions
v0 = 50.0  # initial speed (m/s)
theta_deg = 45.0  # launch angle (degrees)
theta = np.radians(theta_deg)  # convert to radians

# Initial velocity components
vx0 = v0 * np.cos(theta)
vy0 = v0 * np.sin(theta)

# Initial state: [x0, y0, vx0, vy0]
initial_state = [0.0, 0.0, vx0, vy0]

# Drag parameter based on terminal velocity
# For a baseball: v_terminal ≈ 30-40 m/s
v_terminal = 35.0  # m/s
gamma = g / v_terminal**2

# Time span for integration
t_max = 15.0  # maximum time (s)
t_span = (0, t_max)
t_eval = np.linspace(0, t_max, 1000)

print(f"Initial conditions:")
print(f"  Initial speed: v₀ = {v0} m/s")
print(f"  Launch angle: θ = {theta_deg}°")
print(f"  Initial velocity: vₓ₀ = {vx0:.2f} m/s, vᵧ₀ = {vy0:.2f} m/s")
print(f"\nDrag parameters:")
print(f"  Terminal velocity: v_t = {v_terminal} m/s")
print(f"  Drag coefficient: γ = {gamma:.6f} 1/m")

## Numerical Solution

In [ ]:
# Solve ODE with drag
solution_drag = solve_ivp(
    projectile_with_drag,
    t_span,
    initial_state,
    args=(gamma,),
    method='RK45',
    t_eval=t_eval,
    events=hit_ground,
    dense_output=True
)

# Extract results for case with drag
t_drag = solution_drag.t
x_drag = solution_drag.y[0]
y_drag = solution_drag.y[1]
vx_drag = solution_drag.y[2]
vy_drag = solution_drag.y[3]

# Find where y becomes negative (ground impact)
ground_idx = np.where(y_drag < 0)[0]
if len(ground_idx) > 0:
    idx_end = ground_idx[0]
    t_drag = t_drag[:idx_end]
    x_drag = x_drag[:idx_end]
    y_drag = y_drag[:idx_end]
    vx_drag = vx_drag[:idx_end]
    vy_drag = vy_drag[:idx_end]

# Analytical solution without drag
t_flight_ideal = 2 * vy0 / g  # time of flight
t_ideal = np.linspace(0, t_flight_ideal, 500)
x_ideal, y_ideal = projectile_no_drag(t_ideal, v0, theta)

# Calculate key metrics
range_drag = x_drag[-1]
range_ideal = v0**2 * np.sin(2 * theta) / g
max_height_drag = np.max(y_drag)
max_height_ideal = (v0 * np.sin(theta))**2 / (2 * g)
time_flight_drag = t_drag[-1]
time_flight_ideal = t_flight_ideal

print("Trajectory computed successfully.")
print(f"\nResults comparison:")
print(f"{'Quantity':<20} {'With Drag':>12} {'No Drag':>12} {'Reduction':>12}")
print("-" * 58)
print(f"{'Range (m)':<20} {range_drag:>12.2f} {range_ideal:>12.2f} {100*(1-range_drag/range_ideal):>11.1f}%")
print(f"{'Max Height (m)':<20} {max_height_drag:>12.2f} {max_height_ideal:>12.2f} {100*(1-max_height_drag/max_height_ideal):>11.1f}%")
print(f"{'Flight Time (s)':<20} {time_flight_drag:>12.2f} {time_flight_ideal:>12.2f} {100*(1-time_flight_drag/time_flight_ideal):>11.1f}%")

## Visualization

We create a comprehensive visualization showing the trajectory comparison and velocity profiles.

In [ ]:
# Create figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Projectile Motion: Effect of Air Resistance', fontsize=14, fontweight='bold')

# Plot 1: Trajectory comparison
ax1 = axes[0, 0]
ax1.plot(x_ideal, y_ideal, 'b-', linewidth=2, label='No drag (ideal)')
ax1.plot(x_drag, y_drag, 'r-', linewidth=2, label='With drag')
ax1.set_xlabel('Horizontal Distance (m)', fontsize=11)
ax1.set_ylabel('Height (m)', fontsize=11)
ax1.set_title('Trajectory Comparison', fontsize=12)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, None)
ax1.set_ylim(0, None)

# Add annotations
ax1.annotate(f'Range: {range_ideal:.1f} m', 
             xy=(range_ideal, 0), xytext=(range_ideal*0.7, max_height_ideal*0.3),
             arrowprops=dict(arrowstyle='->', color='blue', alpha=0.7),
             fontsize=9, color='blue')
ax1.annotate(f'Range: {range_drag:.1f} m', 
             xy=(range_drag, 0), xytext=(range_drag*0.5, max_height_ideal*0.15),
             arrowprops=dict(arrowstyle='->', color='red', alpha=0.7),
             fontsize=9, color='red')

# Plot 2: Speed vs time
ax2 = axes[0, 1]
speed_drag = np.sqrt(vx_drag**2 + vy_drag**2)
vx_ideal_t = v0 * np.cos(theta) * np.ones_like(t_ideal)
vy_ideal_t = v0 * np.sin(theta) - g * t_ideal
speed_ideal = np.sqrt(vx_ideal_t**2 + vy_ideal_t**2)

ax2.plot(t_ideal, speed_ideal, 'b-', linewidth=2, label='No drag')
ax2.plot(t_drag, speed_drag, 'r-', linewidth=2, label='With drag')
ax2.axhline(y=v_terminal, color='gray', linestyle='--', alpha=0.7, label=f'Terminal velocity ({v_terminal} m/s)')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Speed (m/s)', fontsize=11)
ax2.set_title('Speed vs Time', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Plot 3: Velocity components with drag
ax3 = axes[1, 0]
ax3.plot(t_drag, vx_drag, 'g-', linewidth=2, label='$v_x$ (horizontal)')
ax3.plot(t_drag, vy_drag, 'm-', linewidth=2, label='$v_y$ (vertical)')
ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Velocity (m/s)', fontsize=11)
ax3.set_title('Velocity Components (With Drag)', fontsize=12)
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

# Plot 4: Multiple launch angles with drag
ax4 = axes[1, 1]
angles = [30, 45, 60, 75]
colors = plt.cm.viridis(np.linspace(0, 0.8, len(angles)))

for angle, color in zip(angles, colors):
    theta_i = np.radians(angle)
    vx0_i = v0 * np.cos(theta_i)
    vy0_i = v0 * np.sin(theta_i)
    state_i = [0.0, 0.0, vx0_i, vy0_i]
    
    sol = solve_ivp(
        projectile_with_drag,
        t_span,
        state_i,
        args=(gamma,),
        method='RK45',
        t_eval=t_eval,
        events=hit_ground
    )
    
    x_i = sol.y[0]
    y_i = sol.y[1]
    
    # Trim to positive y
    idx = np.where(y_i < 0)[0]
    if len(idx) > 0:
        x_i = x_i[:idx[0]]
        y_i = y_i[:idx[0]]
    
    ax4.plot(x_i, y_i, color=color, linewidth=2, label=f'{angle}°')

ax4.set_xlabel('Horizontal Distance (m)', fontsize=11)
ax4.set_ylabel('Height (m)', fontsize=11)
ax4.set_title(f'Trajectories at Different Launch Angles (v₀={v0} m/s)', fontsize=12)
ax4.legend(loc='upper right', title='Launch Angle')
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, None)
ax4.set_ylim(0, None)

plt.tight_layout()

# Save the figure
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
print("Figure saved to 'plot.png'")

plt.show()

## Analysis and Discussion

### Key Observations

1. **Reduced Range**: Air resistance significantly reduces the horizontal range of the projectile. The drag force continuously decelerates the projectile, causing it to fall short of the ideal trajectory.

2. **Asymmetric Trajectory**: Unlike the symmetric parabolic path in vacuum, the trajectory with drag is asymmetric. The descent is steeper than the ascent because:
   - On ascent: gravity and drag both act downward
   - On descent: gravity accelerates the projectile while drag decelerates it

3. **Terminal Velocity**: The speed cannot exceed the terminal velocity during descent. This limits the maximum speed the projectile can achieve.

4. **Optimal Launch Angle**: With air resistance, the optimal launch angle for maximum range is less than 45°. This is because:
   - Lower angles result in higher horizontal velocity
   - Higher horizontal velocity means less time in the air
   - Less time in the air means less cumulative drag

### Physical Interpretation

The drag force depends on $v^2$, making it highly nonlinear. At high speeds, even small increases in velocity result in large increases in drag. This is why:

- The horizontal velocity component decreases monotonically
- The speed profile shows rapid initial deceleration followed by slower changes
- The trajectory bends more sharply downward as the projectile slows

### Applications

Understanding projectile motion with air resistance is crucial in:
- **Sports physics**: Baseball, golf, basketball trajectories
- **Military ballistics**: Artillery and small arms fire
- **Aerospace engineering**: Atmospheric reentry, parachute descent
- **Environmental science**: Particle dispersion, seed dispersal

## Conclusion

This notebook demonstrated the significant impact of air resistance on projectile motion through numerical simulation. The key findings are:

1. Air resistance reduces range, maximum height, and flight time
2. The trajectory becomes asymmetric, with a steeper descent
3. The optimal launch angle shifts below 45°
4. Terminal velocity provides a natural speed limit during descent

The numerical approach using `scipy.integrate.solve_ivp` provides accurate solutions to the nonlinear differential equations that cannot be solved analytically. This methodology can be extended to include other effects such as wind, spin (Magnus effect), or altitude-dependent air density.